# Convolutional Neural Networks (convnets or CNNs) with tf.keras and tf.data.

For this notebook we will look at a convnet problem for image classification. Like the first two tutorials, we will classify Fashion-MNIST digits. This tutorial was inspired by [this one](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.1-introduction-to-convnets.ipynb).

In [1]:
import tensorflow as tf
import numpy as np
import time

In [2]:
tf.random.set_seed(42)

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

In [4]:
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

train_images = np.asarray(train_images, dtype=np.float32) / 255

# Convert the train images and add channels
train_images = train_images.reshape((TRAINING_SIZE, 28, 28, 1))

test_images = np.asarray(test_images, dtype=np.float32) / 255
# Convert the train images and add channels
test_images = test_images.reshape((TEST_SIZE, 28, 28, 1))

In [5]:
# How many categories we are predicting from (0-9)
LABEL_DIMENSIONS = 10

train_labels  = tf.keras.utils.to_categorical(train_labels, LABEL_DIMENSIONS)
test_labels = tf.keras.utils.to_categorical(test_labels, LABEL_DIMENSIONS)

# Cast the labels to floats, needed later
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt',
    'Sneaker', 'Bag', 'Ankle boot'
]

In [6]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu, input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


You can see above that the output of every convolutional layer is a 3D tensor of shape (`height`, `width`, `filters`). The width and height tend to get smaller as we go deeper into the network and the number of filters or channels increases from the input channel size of 1. 

The last part of the network for the classification task is similar to the other notebooks and consists of `Dense` layers which process 1D vectors. So we first need to `Flatten` our 3D outputs from the convolutional part to 1D and then add the `Dense` layers:

In [7]:
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(LABEL_DIMENSIONS, activation=tf.nn.softmax))

Training the network is again similar to all the previous notebooks:

In [8]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [9]:
BATCH_SIZE=128

# Because tf.data may work with potentially **large** collections of data
# we do not shuffle the entire dataset by default
# Instead, we maintain a buffer of SHUFFLE_SIZE elements
# and sample from there.
SHUFFLE_SIZE = 10000 

# Create the dataset
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
training_dataset = training_dataset.shuffle(SHUFFLE_SIZE)
training_dataset = training_dataset.batch(BATCH_SIZE).repeat()

validation_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_dataset = validation_dataset.shuffle(SHUFFLE_SIZE)
validation_dataset = validation_dataset.batch(BATCH_SIZE).repeat()

In [10]:
EPOCHS = 14

model.fit(training_dataset, 
    validation_data=validation_dataset,
    steps_per_epoch = 1000 // 32,
    epochs=EPOCHS,
    validation_steps=1000 // 32)

Train for 31 steps, validate for 31 steps
Epoch 1/14
31/31 [==============================] - 4s 114ms/step - loss: 1.4879 - accuracy: 0.4682 - val_loss: 0.9293 - val_accuracy: 0.6815
Epoch 2/14
31/31 [==============================] - 3s 81ms/step - loss: 0.8981 - accuracy: 0.6656 - val_loss: 0.8139 - val_accuracy: 0.7026
Epoch 3/14
31/31 [==============================] - 2s 78ms/step - loss: 0.7699 - accuracy: 0.6996 - val_loss: 0.7061 - val_accuracy: 0.7296
Epoch 4/14
31/31 [==============================] - 2s 76ms/step - loss: 0.6877 - accuracy: 0.7490 - val_loss: 0.6589 - val_accuracy: 0.7513
Epoch 5/14
31/31 [==============================] - 3s 82ms/step - loss: 0.6283 - accuracy: 0.7654 - val_loss: 0.6377 - val_accuracy: 0.7467
Epoch 6/14
31/31 [==============================] - 3s 88ms/step - loss: 0.6159 - accuracy: 0.7692 - val_loss: 0.5816 - val_accuracy: 0.7891
Epoch 7/14
31/31 [==============================] - 2s 79ms/step - loss: 0.5553 - accuracy: 0.7805 - val_loss: 

Again to evaluate the model we need to check the accuracy on unseen or test data:

In [11]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
print('\nTest Model  Loss: %.6f\tAccuracy: %.6f' % (test_loss, test_acc))


Test Model  Loss: 0.458993	Accuracy: 0.831500
